In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

In [4]:
years = np.arange(2015, 2025)  # 10 years: 2015 to 2024
revenue = np.array([110, 125, 133, 146, 158, 172, 187, 196, 210, 233], dtype=float)
revenue = revenue.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0,1))
scaled_revenue = scaler.fit_transform(revenue)
window_size = 3

print({'years' : years, 'revenue': revenue, 'scaler':scaler, 'scaled_revenue':scaled_revenue})

{'years': array([2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]), 'revenue': array([[110.],
       [125.],
       [133.],
       [146.],
       [158.],
       [172.],
       [187.],
       [196.],
       [210.],
       [233.]]), 'scaler': MinMaxScaler(), 'scaled_revenue': array([[0.        ],
       [0.12195122],
       [0.18699187],
       [0.29268293],
       [0.3902439 ],
       [0.50406504],
       [0.62601626],
       [0.69918699],
       [0.81300813],
       [1.        ]])}


In [7]:
x_train, y_train = [], []
x_train = list(x_train)  # Shape: (samples, time_steps, features)
y_train = list(y_train)  # Shape: (samples, 1)

for i in range(len(scaled_revenue) - window_size):
    x_train.append(scaled_revenue[i:i + window_size])
    y_train.append(scaled_revenue[i + window_size])

# Convert to numpy arrays
X_train = np.array(x_train)  # Shape: (samples, time_steps, features)
y_train = np.array(y_train)  # Shape: (samples, 1)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

X_train shape: (7, 3, 1)
y_train shape: (7, 1)


In [8]:
model = Sequential([
    LSTM(50, activation='relu', input_shape=(window_size, 1)),
    Dense(1)  # Output layer: one predicted value (regression uses linear activation by default)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

d:\web_design\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        10,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,451 (40.82 KB)

 Trainable params: 10,451 (40.82 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
history = model.fit(X_train, y_train, epochs=200, verbose=0)
print("Training completed.")

Training completed.


In [10]:
predictions = []
current_window = scaled_revenue[-window_size:].reshape(1, window_size, 1)

In [18]:
for _ in range(5):
    next_pred = model.predict(current_window)
    predictions.append(next_pred[0, 0])
    
    # Update the current window: remove the first element and add the new prediction at the end
    current_window = np.append(current_window[:, 1:, :], [[[next_pred[0, 0]]]], axis=1)

# Inverse-transform the predictions to original scale
predicted_revenue = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))
print("Predicted revenue for next 5 years:", predicted_revenue.flatten())


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted revenue for next 5 years: [249.46803 249.46803 249.46803 249.46803 249.46803 249.46803 249.46803
 249.46803 249.46803 249.46803 249.46803 249.46803 275.34274 309.36923
 351.42285 415.12326]
